<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install syft

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import syft as sy

# Create a virtual PyGrid network
hook = sy.TorchHook(torch)

# Define virtual workers
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")

# Simple neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        return torch.sigmoid(self.fc(x))

model = Net()

# Distribute data to workers
data_alice = torch.tensor([[0, 0], [1, 1]], dtype=torch.float).send(alice)
target_alice = torch.tensor([[0], [1]], dtype=torch.float).send(alice)

data_bob = torch.tensor([[1, 0], [0, 1]], dtype=torch.float).send(bob)
target_bob = torch.tensor([[1], [1]], dtype=torch.float).send(bob)

optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(10):
    # Training on Alice's data
    model.send(alice)
    optimizer.zero_grad()
    pred_alice = model(data_alice)
    loss_alice = ((pred_alice - target_alice) ** 2).sum()
    loss_alice.backward()
    optimizer.step()
    model.get()

    # Training on Bob's data
    model.send(bob)
    optimizer.zero_grad()
    pred_bob = model(data_bob)
    loss_bob = ((pred_bob - target_bob) ** 2).sum()
    loss_bob.backward()
    optimizer.step()
    model.get()

    print(f"Epoch {epoch + 1}, Loss: {loss_alice.item() + loss_bob.item()}")